In [1]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.6 MB/s eta 0:00:00


In [2]:
#Importing necessary libraries
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import Word
from textblob import TextBlob
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#read in file
file_path = '/content/drive/MyDrive/proj4-repo/data/cleaned_data/cleaned_training_tweets.csv'

df = pd.read_csv(file_path)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/data/cleaned_data/cleaned_training_tweets.csv'

In [ ]:
#read in file
#uploaded = files.upload()
#pd

#df = pd.read_csv('cleaned_training_tweets.csv')
#df.head()

Saving cleaned_training_tweets.csv to cleaned_training_tweets.csv


,id,user,date,text,target
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,0
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,0
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,0
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",0
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,0


# Copy the DataFrame to test both TextBlob and VaderSentiment

In [15]:
original_df = df.copy()

In [16]:
original_df = original_df.drop(columns=['id', 'user', 'date'])
original_df.head()

,text,target
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0


In [17]:
text_blob_df = df.copy()

In [18]:
#drop the target column
text_blob_df = text_blob_df.drop(columns=['target', 'date', 'id', 'user'])
text_blob_df.head()

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew


In [19]:
vader_sentiment_df = df.copy()

In [20]:
#drop the target column
vader_sentiment_df = vader_sentiment_df.drop(columns=['target', 'date', 'id', 'user'])
vader_sentiment_df.head()

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew


# Original data LSTM(Long Short-Term Memory Model)

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Pre-Processing the text
def cleaning(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word.isalpha()])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]
    return ' '.join(lemmatized_text)

original_df['cleaned_text'] = original_df['text'].apply(cleaning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [22]:
# Encoding labels
label_encoder = LabelEncoder()
df['target'] = label_encoder.fit_transform(df['target'])
labels = to_categorical(df['target'])

In [24]:
# Tokenization and Padding
max_features = 10000  # Limiting features for efficiency
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(original_df['cleaned_text'].values)
X = tokenizer.texts_to_sequences(original_df['cleaned_text'].values)
X = pad_sequences(X)

In [25]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [26]:
# Model Building
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
# Model Summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 33, 128)           1280000   
                                                                 
 spatial_dropout1d (Spatial  (None, 33, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1707394 (6.51 MB)
Trainable params: 1707394 (6.51 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [29]:
# Model Training
epochs = 20
batch_size = 300
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
4267/4267 [==============================] - 512s 120ms/step - loss: 0.5085 - accuracy: 0.7468 - val_loss: 0.4931 - val_accuracy: 0.7575
Epoch 2/20
4267/4267 [==============================] - 498s 117ms/step - loss: 0.4884 - accuracy: 0.7598 - val_loss: 0.4898 - val_accuracy: 0.7597
Epoch 3/20
4267/4267 [==============================] - 493s 115ms/step - loss: 0.4795 - accuracy: 0.7656 - val_loss: 0.4870 - val_accuracy: 0.7613
Epoch 4/20
4267/4267 [==============================] - 502s 118ms/step - loss: 0.4716 - accuracy: 0.7707 - val_loss: 0.4871 - val_accuracy: 0.7611
Epoch 5/20
4267/4267 [==============================] - 499s 117ms/step - loss: 0.4639 - accuracy: 0.7752 - val_loss: 0.4875 - val_accuracy: 0.7614
Epoch 6/20
4267/4267 [==============================] - 516s 121ms/step - loss: 0.4564 - accuracy: 0.7796 - val_loss: 0.4916 - val_accuracy: 0.7607
Epoch 7/20
4267/4267 [==============================] - 497s 117ms/step - loss: 0.4487 - accuracy: 0.7839 - val_

In [30]:
# Model Testing
model_loss, model_accuracy = model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10000/10000 - 107s - loss: 0.5631 - accuracy: 0.7517 - 107s/epoch - 11ms/step
Loss: 0.5630591511726379, Accuracy: 0.7516937255859375


In [31]:
# Model Testing
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

Test Loss: 0.5630591511726379
Test Accuracy: 0.7516937255859375


In [33]:
#save Model
model.save('original_df_model.keras')


In [35]:
#Download Model file
from google.colab import files
files.download('original_df_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TextBlob Sentiment analysis

In [ ]:
# Define a function to perform sentiment analysis using TextBlob and convert to labels
def analyze_sentiment_blob(text):
    analysis = TextBlob(text)
    sentiment_score = analysis.sentiment.polarity
    if sentiment_score > 0.2:
        return 'positive'
    elif sentiment_score < -0.2:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to the 'text' column and create a new column 'sentiment_label'
text_blob_df['textblob_sentiment_label'] = text_blob_df['text'].apply(analyze_sentiment_blob)
text_blob_df.head()

,id,user,date,text,textblob_sentiment_label
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,neutral
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,positive
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,neutral
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",negative
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,neutral


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Pre-Processing the text
def cleaning(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word.isalpha()])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]
    return ' '.join(lemmatized_text)

text_blob_df['cleaned_text'] = text_blob_df['text'].apply(cleaning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
text_blob_df.head()

,id,user,date,text,textblob_sentiment_label,cleaned_text
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,neutral,upset update facebook texting might cry result...
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,positive,dived many time managed save rest go bound
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,neutral,whole body feel itchy like fire
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",negative,behaving see
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,neutral,whole crew


In [ ]:
# Tokenization and Padding
max_features = 10000  # Limiting features for efficiency
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(text_blob_df['cleaned_text'].values)
X = tokenizer.texts_to_sequences(text_blob_df['cleaned_text'].values)
X = pad_sequences(X)

In [ ]:
#Encode Sentiment Labels
label_encoder = LabelEncoder()
text_blob_df['sentiment_label_encoded'] = label_encoder.fit_transform(text_blob_df['textblob_sentiment_label'])
y = text_blob_df['sentiment_label_encoded'].values

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [ ]:
# Model Building
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Model Summary
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 33, 128)           1280000   
                                                                 
 spatial_dropout1d_2 (Spati  (None, 33, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 256)               394240    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1707394 (6.51 MB)
Trainable params: 1707394 (6.51 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Model Training
epochs = 20
batch_size = 500
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
2560/2560 [==============================] - 320s 124ms/step - loss: 0.5118 - accuracy: 0.7440 - val_loss: 0.4952 - val_accuracy: 0.7554
Epoch 2/20
2560/2560 [==============================] - 300s 117ms/step - loss: 0.4919 - accuracy: 0.7578 - val_loss: 0.4907 - val_accuracy: 0.7588
Epoch 3/20
2560/2560 [==============================] - 299s 117ms/step - loss: 0.4845 - accuracy: 0.7622 - val_loss: 0.4888 - val_accuracy: 0.7600
Epoch 4/20
2560/2560 [==============================] - 298s 117ms/step - loss: 0.4787 - accuracy: 0.7658 - val_loss: 0.4883 - val_accuracy: 0.7612
Epoch 5/20
2560/2560 [==============================] - 298s 116ms/step - loss: 0.4737 - accuracy: 0.7691 - val_loss: 0.4880 - val_accuracy: 0.7618
Epoch 6/20
2560/2560 [==============================] - 302s 118ms/step - loss: 0.4684 - accuracy: 0.7723 - val_loss: 0.4886 - val_accuracy: 0.7617
Epoch 7/20
2560/2560 [==============================] - 301s 117ms/step - loss: 0.4640 - accuracy: 0.7749 - val_

In [ ]:
# Model Testing
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

Test Loss: 0.5297048091888428
Test Accuracy: 0.7563218474388123


# VaderSentiment analysis

In [ ]:
sentiment = SentimentIntensityAnalyzer()
#create a function to apply to our data frame
def analyze_sentiment_vader (text):
    sentiment_score = sentiment.polarity_scores(text)
    return sentiment_score['compound']
#apply the created function to the copied vader_sentiment_df data frame
vader_sentiment_df['sentiment_vader'] = vader_sentiment_df['text'].apply(analyze_sentiment_vader)
vader_sentiment_df.head()

,id,user,date,text,sentiment_vader
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,-0.7500
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,0.4939
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,-0.2500
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",-0.4939
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,0.0000


In [ ]:
 #Convert Sentiment Scores to Categorical Labels
def convert_sentiment_vader(score):
    if score > 0.2:
        return 'positive'
    elif score < -0.2:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to create a new column 'sentiment_label'
vader_sentiment_df['vader_sentiment_label'] = vader_sentiment_df['sentiment_vader'].apply(convert_sentiment_vader)
vader_sentiment_df.head()

,id,user,date,text,sentiment_vader,vader_sentiment_label
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,-0.7500,negative
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,0.4939,positive
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,-0.2500,negative
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",-0.4939,negative
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,0.0000,neutral


In [ ]:
vader_sentiment_df = vader_sentiment_df.drop(columns=['sentiment_vader'])
vader_sentiment_df.head()

,id,user,date,text,vader_sentiment_label
0,1467810672,scotthamilton,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,negative
1,1467810917,mattycus,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,positive
2,1467811184,ElleCTF,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,negative
3,1467811193,Karoli,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",negative
4,1467811372,joy_wolf,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,neutral


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Pre-Processing the text
def cleaning(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word.isalpha()])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]
    return ' '.join(lemmatized_text)

vader_sentiment_df['cleaned_text'] = vader_sentiment_df['text'].apply(cleaning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Tokenization and Padding
max_features = 10000  # Limiting features for efficiency
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(vader_sentiment_df['cleaned_text'].values)
X = tokenizer.texts_to_sequences(vader_sentiment_df['cleaned_text'].values)
X = pad_sequences(X)

In [ ]:
#Encode Sentiment Labels
label_encoder = LabelEncoder()
vader_sentiment_df['sentiment_label_encoded'] = label_encoder.fit_transform(vader_sentiment_df['vader_sentiment_label'])
y = vader_sentiment_df['sentiment_label_encoded'].values

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [ ]:
# Model Building
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Model Summary
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 33, 128)           1280000   
                                                                 
 spatial_dropout1d_3 (Spati  (None, 33, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_3 (LSTM)               (None, 256)               394240    
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1707394 (6.51 MB)
Trainable params: 1707394 (6.51 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Model Training
epochs = 20
batch_size = 500
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
2560/2560 [==============================] - 314s 121ms/step - loss: 0.5118 - accuracy: 0.7442 - val_loss: 0.4951 - val_accuracy: 0.7558
Epoch 2/20
2560/2560 [==============================] - 297s 116ms/step - loss: 0.4919 - accuracy: 0.7576 - val_loss: 0.4903 - val_accuracy: 0.7598
Epoch 3/20
2560/2560 [==============================] - 295s 115ms/step - loss: 0.4844 - accuracy: 0.7624 - val_loss: 0.4887 - val_accuracy: 0.7603
Epoch 4/20
2560/2560 [==============================] - 294s 115ms/step - loss: 0.4786 - accuracy: 0.7660 - val_loss: 0.4869 - val_accuracy: 0.7613
Epoch 5/20
2560/2560 [==============================] - 294s 115ms/step - loss: 0.4733 - accuracy: 0.7695 - val_loss: 0.4885 - val_accuracy: 0.7621
Epoch 6/20
2560/2560 [==============================] - 295s 115ms/step - loss: 0.4683 - accuracy: 0.7722 - val_loss: 0.4878 - val_accuracy: 0.7620
Epoch 7/20
2560/2560 [==============================] - 295s 115ms/step - loss: 0.4634 - accuracy: 0.7755 - val_

In [ ]:
# Model Testing
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

NameError: name 'model' is not defined